## Basic workflow in langchain

1. Define the LLM wrapper 
2. Define a chat template (or prompt)
3. Create a chain using the prompt and the llm wrapper
4. Invoke the chain. The chain is what does all the work

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")
llm.invoke("Do pigs fly?")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class story teller."),
    ("user", "{input}")
])

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [ ]:
story = chain.invoke(
    {"input": "Write a story in five lines about a pig happily flying in the sky."})
print(story)

## Retrieval

1. Find a document source
2. Index doc into a vector store
3. Load doc embeddings
4. Create smaller chunks from large docs
5. Create a retrieval chain
6. Invoke

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jujutsu-kaisen.fandom.com/wiki/Yuji_Itadori")

docs = loader.load()

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke(
    {"input": "What is Divergent Fist?"})
print(response["answer"])

In [ ]:
response = retrieval_chain.invoke(
    {"input": "What is Black Flash?"})
print(response["answer"])